# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tuy hoa,13.0833,109.3000,26.28,85,55,2.63,VN,1701827803
1,1,hadibu,12.6500,54.0333,24.68,62,51,6.81,YE,1701827252
2,2,hawaiian paradise park,19.5933,-154.9731,27.16,85,0,4.63,US,1701827803
3,3,shikhany,52.1167,47.2167,-12.27,99,68,1.61,RU,1701827804
4,4,albany,42.6001,-73.9662,0.50,88,100,1.31,US,1701827696


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
City_Map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
City_Map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df.loc[
    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
new_ideal_city_data_df = ideal_city_data_df.dropna()

# Display sample data
new_ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
49,49,mendoza,-32.8908,-68.8272,23.74,50,0,0.00,AR,1701827812
57,57,bouna,9.2667,-3.0000,24.09,20,0,0.89,CI,1701827814
77,77,isla vista,34.4133,-119.8610,21.92,43,0,3.09,US,1701827817
87,87,ayorou,14.7308,0.9174,23.62,18,0,2.79,NE,1701827818
180,180,sur,22.5667,59.5289,23.25,44,0,3.75,OM,1701827838
215,215,freetown,8.4840,-13.2299,26.22,82,0,2.33,SL,1701827272
228,228,matoury,4.8483,-52.3317,24.98,94,0,1.54,GF,1701827846
319,319,bartica,6.4000,-58.6167,25.18,95,0,1.25,GY,1701827864
426,426,guisa,20.2550,-76.5386,21.24,96,0,1.09,CU,1701827881
466,466,miragoane,18.4423,-73.0876,26.51,74,0,1.00,HT,1701827889


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(
    {
        "City" : new_ideal_city_data_df["City"],
        "Country" : new_ideal_city_data_df["Country"],
        "Lat" : new_ideal_city_data_df["Lat"],
        "Lng" : new_ideal_city_data_df["Lng"],
        "Humidity" : new_ideal_city_data_df["Humidity"]
    }
)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
49,mendoza,AR,-32.8908,-68.8272,50,
57,bouna,CI,9.2667,-3.0000,20,
77,isla vista,US,34.4133,-119.8610,43,
87,ayorou,NE,14.7308,0.9174,18,
180,sur,OM,22.5667,59.5289,44,
215,freetown,SL,8.4840,-13.2299,82,
228,matoury,GF,4.8483,-52.3317,94,
319,bartica,GY,6.4000,-58.6167,95,
426,guisa,CU,20.2550,-76.5386,96,
466,miragoane,HT,18.4423,-73.0876,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : "accommodation.hotel",
    "apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mendoza - nearest hotel: Hotel Eben-Ezer
bouna - nearest hotel: Hôtel Éléphant  Bouna
isla vista - nearest hotel: The Club & Guest House
ayorou - nearest hotel: Hotel Aminokal
sur - nearest hotel: Sur Hotel
freetown - nearest hotel: Formal Leona hotel
matoury - nearest hotel: La Marmotte
bartica - nearest hotel: Rockville Hotel
guisa - nearest hotel: No hotel found
miragoane - nearest hotel: Chez Dem
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
santa ana - nearest hotel: Holiday Inn Express


,City,Country,Lat,Lng,Humidity,Hotel Name
49,mendoza,AR,-32.8908,-68.8272,50,Hotel Eben-Ezer
57,bouna,CI,9.2667,-3.0000,20,Hôtel Éléphant Bouna
77,isla vista,US,34.4133,-119.8610,43,The Club & Guest House
87,ayorou,NE,14.7308,0.9174,18,Hotel Aminokal
180,sur,OM,22.5667,59.5289,44,Sur Hotel
215,freetown,SL,8.4840,-13.2299,82,Formal Leona hotel
228,matoury,GF,4.8483,-52.3317,94,La Marmotte
319,bartica,GY,6.4000,-58.6167,95,Rockville Hotel
426,guisa,CU,20.2550,-76.5386,96,No hotel found
466,miragoane,HT,18.4423,-73.0876,74,Chez Dem


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)


# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)